# Segmentacja obrazów

## Cel ćwiczenia
- zapoznanie z metodami segmentacji obrazów:
    - segmentacją przez rozrost obszaru (*region growing*)
    - i w ramach zadania domowego segmentacją przez podział i łączenie (*split and merge*)

## Wstęp

W ramach dotychczas wykonanych ćwiczeń poznaliśmy segmentację z wykorzystaniem binaryzacji (progowania) - tj. na podstawie jasności (koloru) poszczególnych pikseli.
Wykonaliśmy dwa warianty metody: globalny i lokalny oraz przetestowaliśmy różne podejścia do automatycznego wyznaczania progu bianryzacji (iteracyjne oraz Otsu).
Ponadto poznaliśmy możliwość segmentacji na podstawie krawędzi z wykorzystaniem transformaty Hougha.

## Podstawy

Niech $R$ oznacza obszar równy całemu analizowanemu obrazowi.
Segmentację możemy uznać za proces podziału $R$ na $n$ podobszarów $R_1,R_2,...,R_n$ takich że:
1. $\cup_{i=1}^n R_i = R$
2. $R_i$ - składa się z połączonych ze sobą pikseli,
3. $R_i \cap R_j = \varnothing $ dla wszystkich $i$ i $j$,$ i \neq j$,
4. $Q(R_i) = TRUE$ dla $i = 1,2,...n$
5. $ Q(R_i \cup R_j) = FALSE$ dla każdych sąsiednich $R_i$ i $R_j$.

gdzie: symbole $\cup$ i $\cap$ oznaczają odpowiednio sumę i iloczyn zbiorów, a $Q$ jest pewnym predykatem.

Punkt *1* oznacza, że segmentacja musi być kompletna tj. każdy piksel powinien zostać przyporządkowany do jakiegoś zbioru.

Punkt *2* oznacza, że piksele w ramach jednego podobszaru muszą być ze sobą połączone (na zasadzie sąsiedztwa 4 lub 8 punktowego).

Punkt *3* oznacza, że dowolne różne podobszary muszą być rozłączne.

Punkt *4* oznacza, że wszystkie piksele będące w ramach jednego podobszaru muszą spełniać pewną własność. Przykładowo może to być ten sam lub podobny odcień szarości.

Punkt *5* oznacza, że dwa sąsiednie podobszary muszą być różne w sensie predykatu Q (inaczej powinny zostać uznane za ten sam podobszar).

## Segmentacja przez rozrost obszaru

Pomysł jest następujący.
Wybieramy (jak ? - o tym później) piksele startowe (ang. *seed*) i od nich zaczynamy segmentację.
Odbywa się ona na zasadzie sprawdzania czy sąsiednie piksele (sąsiedztwo 4 lub 8 punktowe) są podobne do centralnego pod względem jakieś cechy (predykatu $Q$).
Jeśli tak to oznaczane są jako należące do tej samej klasy co piksel centralny.
Ponadto stają się one kolejnymi punktami startowymi metody.
Zatem procedura ma charakter rekurencyjny.

Wybór punktów startowych może być podyktowany charakterem problemu (przykładowo wiemy gdzie na pewno zaczynają się obiekty).
W ogólnym przypadku trzeba założyć, że pikselem startowym może być każdy piksel, co oczywiście wpływa na złożoność metody.

Kolejnym problemem jest wybór kryterium stopu tj. kiedy nasza procedura rekurencyjna ma się zakończyć.
Dla danego podobszaru będzie to moment, kiedy nie istnieją już piksele, które można do  niego dołączyć.

Warto w tym miejscu zwrócić uwagę, że stosowanie ''sztywnego'' warunku - np. różnica jasności pomiędzy pikselem centralnym, a analizowanym jest mniejsza niż 5 - może często dać niepożądane wyniki, gdyż nie uwzględnia pewnych lokalnych właściwości.
Przykładowo, może się okazać, że jeśli na obrazie występuje niewielki gradient to za należące do tego samego obszaru uznane zostaną piksele o zupełnie różnych jasnościach.
Możliwa jest też sytuacja odwrotna.
Duże zróżnicowanie wartości na obrazie spowoduje zbyt duże ''poszarpanie'' wykrytych obszarów.

Jednym z możliwych rozwiązań jest uzależnienie kryterium podobieństwa (predykatu $Q$) od własności obrazu np. średniej jasności w obrębie danego obszaru.
Można również dodać inne kryteria np. kształt podobszaru itp.

Uwaga. Pojęcie segmentacja przez rozrost to pewna **koncepcja** podejścia do segmentacji, a nie konkretna metoda.
Na etapie projektowania algorytmu należy skupić się na konstrukcji kryterium podobieństwa (tj. co i jak ma być ze sobą porównywane) oraz ewentualnym uzupełnianiu metody o dodatkowe kryteria.



## Zadanie: zaprojektować system segmentacji wybranej struktury na obrazie MRI (np. stawu kolanowego).
Punkt startowy wyznaczany będzie ''ręcznie'' (poprzez kliknięcie na obrazie).

1. Wczytaj obraz *knee.png* (w skali szarości) - MRI stawu kolanowego. Wyświetl go. Załóżmy, że chcemy dokonać segmentacji górnej kości. Przyjęliśmy, że punkt startowy metody wyznaczany będzie w sposób ręczny. Do pobrania położenia kursora myszy na ekranie służy funkcja `plt.ginput`. Uwaga. Jeśli obrazki są ''osadzone'' w notatniku, funkcja nie działa - proszę zwrócić uwagę na kod `matplotlib.use('TkAgg')`, który powoduje, że obrazki wyświetlane są ''samodzielnie''.

  Uwaga 1. Pobrane współrzędne należy zaokrąglić (*floor* lub *round*).

  Uwaga 2. Dla potrzeb testów dobrze jest wpisać punkt startowy na ''sztywno''. Pozwoli to uniknąć dość irytującego klikania przy każdym uruchomieniu skryptu.


2. Metodę zaimplementujemy z wykorzystaniem stosu.
   Uwaga. Podany poniżej opis jest tylko jedną z możliwych realizacji (niekoniecznie najlepszą).
   Na początek tworzymy dwie macierze o rozmiarach takich jak analizowany obraz.
   W jednej będziemy zapisywać odwiedzone lokalizacje (`visited` - typ *boolean*), a w drugiej rezultaty segmentacji (`segmented`).
   Obie macierze tworzymy wypełnione zerami (funkcja `np.zeros`).
   Tworzymy też stos - w Python to po prostu ''pusta lista''.

3. W pierwszym kroku metody na stos (`stack.append`) odkładamy współrzędne wybranego przez użytkowania piksela.
   Oznaczamy go również jako odwiedzony (macierz *visited*) i zaliczony do obiektu (macierz *segmented*).

4. Pozostałe działanie odbywać się będzie w pętli `while`, której warunkiem stopu jest obecność elementów na stosie (`len(stack)>0`).
 W iteracji należy pobrać współrzędne piksela ze stosu.
  Następnie sprawdzamy, czy dla tego piksela można określić kontekst o rozmiarze $ 3 \times 3$ tj. czy ma wszystkich sąsiadów.
  Uwaga. Przyjmujemy tutaj uproszczenie - nie segmentujemy brzegu obrazka (ramki o szerokości 1 piksela).

5. W kolejnym kroku rozpisujemy pętlę po otoczeniu $3 \times 3$ (x2 `for`).
   Wewnątrz obliczamy odległość pomiędzy pikselem centralnym, a każdym z kontekstu.
   Przyjmijmy, że będzie to moduł z różnicy jasności.
   Jeśli wartość modułu będzie mniejsza od zdefiniowanego progu (proszę przyjąć jako początkową wartość 4) oraz rozpatrywany piksel nie był wcześniej odwiedzany to oznaczamy go jako należący do obiektu oraz jego współrzędne ''odkładamy'' na stosie.
   Uwaga. Pierwsza część warunku logicznego to nasz predykat Q.
   Za piksele podobne uznajemy takie, których różnica w jasności jest mniejsza niż zadany próg.
   Niezależnie od wyniku testu oznaczamy piksel jako odwiedzony (żeby wielokrotnie nie analizować tych samych lokalizacji).

6. Poza pętlą `while` proszę wyświetlić rezultat segmentacji.
   Czy wyniki są poprawne ?
   Proszę poeksperymentować z wartością progu.

In [ ]:
import os
import cv2
import matplotlib
#matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
import numpy as np

if not os.path.exists("knee.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/12_Segmentation/knee.png --no-check-certificate

I_knee = cv2.imread('knee.png', cv2.IMREAD_GRAYSCALE)
        
plt.imshow(I_knee, 'gray')
plt.show()

In [ ]:

def segmentation_global(image, x, y, threshold=4):
    X,Y = image.shape
    #odwiedzone
    visited = np.zeros(image.shape, dtype=np.uint64)
    #segmentacja
    segmented = np.zeros(image.shape, dtype=np.uint64)
    #stos
    stack = []
    stack.append((x,y))
    #odwiedzony
    visited[x,y] = 1
    segmented[x,y] = 1
    #warunek obcności elementów na stosie
    while len(stack) > 0:
        #pobranie ze stosu
        pkt = stack.pop()
        x2 = pkt[0]
        y2 = pkt[1]
        
        for i in range(3):
            for j in range(3):
                #moduł z róznicy jasności
                odl = np.abs(image[x2-1+i,y2-1+j] - image[x,y])
            if (threshold > odl) and (visited[x2-1+i,y2-1+j] == 0):
                stack.append((x2-1+i,y2-1+j))
                segmented[x2-1+i,y2-1+j] = image[x2-1+i,y2-1+j]
            visited[x2-1+i,y2-1+j] = 1
         
    return segmented        

p1 = segmentation_global(I_knee,200, 250,60)
fig, ax = plt.subplots()
fig.set_size_inches(7, 7)
ax.imshow(p1, 'gray')
ax.axis('off')
plt.show()

p2 = segmentation_global(I_knee,100, 50, 60)
fig, ax = plt.subplots()
fig.set_size_inches(7, 7)
ax.imshow(p2, 'gray')
ax.axis('off')
plt.show()

p3 = segmentation_global(I_knee,300, 280, 60)
fig, ax = plt.subplots()
fig.set_size_inches(7, 7)
ax.imshow(p3, 'gray')
ax.axis('off')
plt.show()

p4 = segmentation_global(I_knee,350, 200, 60)
fig, ax = plt.subplots()
fig.set_size_inches(7, 7)
ax.imshow(p4, 'gray')
ax.axis('off')
plt.show()

In [ ]:
#Komentarz
#Wyniki wydają się być poprawne. Wybrany przeze mnie próg może skutkować tym, że w niektórych sytuacjach w zależności od 
#punktu startowego obrazek może być poszarpany.

1. Powyższy przykład ukazuje wspomniany wcześniej problem z ''globalnym'' podejściem do predykatu Q.
  Jeśli próg będzie mały, to wyznaczymy jedynie niewielki fragment kości.
  Natomiast zwiększenie progu skutkuje segmentacją nadmiarową.
  Mówiąc kolokwialnie, na obrazie znajdzie się ''ścieżka'' po której możliwe jest przejście od obszaru jasnego do ciemnego nie ''łamiąc'' progu odległości pomiędzy sąsiednimi pikselami.

2. Aby zaradzić powyższemu problemowi, można za kryterium podobieństwa przyjąć, nie różnicę jasności względem piksela centralnego, a od globalnie wyznaczonego i aktualizowanego progu.
  W najprostszym przypadku może to być średnia jasność w wyznaczonym obszarze.
  W celu implementacji mechanizmu wystarczy dodać dwie zmienne: średnią ($mV$) oraz licznik pikseli uznanych za należące do obiektu ($nS$).
  Przy każdym zdjęciu ze stosu licznik jest zwiększany o 1.
  Aktualizacja średniej następuje na podstawie równania:
  \begin{equation}
  mV_{n} = \frac{mV_{nS-1} (nS - 1) + I}{nS}
  \tag{1}
  \end{equation}
  
  Następnie wystarczy tylko zmienić sposób obliczania odległości - zamienić piksel centralny na wartość średnią.
  Proszę spróbować jak działa metodą z taką modyfikacją - proszę się liczyć z koniecznością zwiększenia progu (nawet dość znaczną).

3. Poprawić działanie metody może również dodanie filtracji uśredniającej np. filtrem Gaussa.

In [ ]:
knee_1 = cv2.GaussianBlur(I_knee,(5,5),0)
def segmentation_lokal(image, x, y, threshold = 4):
  srednia = image[x,y]
  srednia_n = 0
  visited = np.zeros(image.shape)
  segmented = np.zeros(image.shape)
  stack = []
  stack.append((x,y))
  visited[x,y] = 1
  segmented[x,y] = 1
  while len(stack) > 0:
    pkt = stack.pop()
    x2 = pkt[0]
    y2 = pkt[1]
    if srednia_n != 0:
        srednia = (srednia * (srednia_n-1) + image[x2,y2]) / srednia_n
    srednia_n = srednia_n + 1;
    for i in range(3):
      for j in range(3):
        odl = np.abs(image[x2-1+i,y2-1+j] - srednia)
        if (threshold > odl) and (visited[x2-1+i,y2-1+j] == 0):
          stack.append((x2-1+i,y2-1+j))
          segmented[x2-1+i,y2-1+j] = image[x2-1+i,y2-1+j]
        visited[x2-1+i,y2-1+j] = 1
  return segmented

p = segmentation_lokal(I_knee, 200, 250, 25)
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
ax.imshow(p, 'gray')
ax.set_title('Przed filtracją')
ax.axis('off')
plt.show()



In [ ]:
knee_1 = cv2.GaussianBlur(I_knee,(5,5),0)
plt.imshow(knee_1,'gray')
plt.show()

In [ ]:
p1 = segmentation_lokal(knee_1, 200, 250, 60)
fig, ax = plt.subplots()
fig.set_size_inches(10, 10)
ax.imshow(p1, 'gray')
ax.axis('off')
ax.set_title('Po dodaniu filtracji')
plt.show()



In [ ]:
p5 = segmentation_lokal(knee_1, 550, 550, 60)
p4 = segmentation_lokal(knee_1, 300, 380, 60)
p2 = segmentation_lokal(knee_1, 350, 350, 60)
p3 = segmentation_lokal(knee_1, 400, 450, 60)
fig, axs = plt.subplots(2, 2)
fig.set_size_inches(20, 20)
fig.suptitle('Po dodaniu filtracji dla różnych punktów startowych', fontsize=30)
axs[0,0].imshow(p4, 'gray')
axs[0,0].axis('off')
axs[0,1].imshow(p2, 'gray')
axs[0,1].axis('off')
axs[1,0].imshow(p3, 'gray')
axs[1,0].axis('off')
axs[1,1].imshow(p5, 'gray')
axs[1,1].axis('off')
plt.show()
